# Welcome
This is the second part of the series on NLP which is specially signifies the working of the `Attention Mechanism` in `Keras` for Natural Language Processing. If you want to start the NLP  or Deep journey, I have few notebooks for your use and please do watch then in sequence.

1. [Deep Learning Beginner Tutorial (Using CNN with Keras)](https://www.kaggle.com/deshwalmahesh/bengali-ai-complete-beginner-tutorial-95-acc)
2. [NLP Tutorial for Beginner describing Embeddings, RNN, LSTM, GRU](https://www.kaggle.com/deshwalmahesh/nlp-beginner-1-rnn-lstm-gru-embeddings-glove)

**NOTE**: This notebook will use of the custom `Layer` in Keras as there is no `Attention` Layer given in the official package so if you do not how to make a custom layer, please find this very intutive notebook about [cutom layers, models in keras](https://www.kaggle.com/deshwalmahesh/keras-tensorflow-2-0-custom-layers-models)



# Problem
We have been given full articles from the BBC where these articles can belong to one of the 5 categories. We have to look at a new article and find which category does it belong to.


# Solution
We'll won't make use of the sentiments here. Idea is very simple that there can be some words like `cricket`, `football` ,`goals`, `runs` and so on which can define the category of the news. Same goes with science and other genres as well. We'll try to build a modle which is able to predict these things for us.

# Attention Here!!!
So what basically attention is? Why is it used? How is it used?......
Well, to answer your question regarding attention, we have to **Roll Back in Time** (pun intended). Before the use of Attention in NLP tasks, we relied heavily on `LSTM` which used to **Roll Back in Time** to get the context and sementic of a word in a sentence. So what exactly is rolling back?? As we know that any `RNN` be it `GRU` or even `LSTM`, produces a hiden state `H_i` with every time step. Sate `H_i+1` defined that it has learned from the `Input + H_i`. So the new state has all the **Important** information needed up until now i.e it has all the information learned by previous cells, filtered it and read the new input to produce a new Hidden state which it'll pass to the next cell.

This is all fun and games until it's something with a deep meaning. LSTMs are good at finding relation when it comes to short sentences but what about a time when you have to find the hidden meaning? Now I got your Attention!!! or you can say we got Attention. Attention is very efficient at finding the hidden meanings of the words as well as the context of the word. A very popular example you'll see everywhere will be this one.
<img src="https://i.stack.imgur.com/YNdXP.png" width="350px">

So What is this saying?

It is saying that the `Attention` Mechanism is able to get the part of English language that **it** is used in the context for **Animal** and it has decided to give this decision based after looking at the terms and making decision that  **The,animal,didn't** are more important than the **the,street,too**. So if you had change the last word from `tired -> narrow`, it would hve looked at `the,street,too` more than `the,animal,didn't` because it would have learned that **it** here is used for street but not the animal.

GREATTTT!!!! But how does it do that??? 
Well it is doing by computing **self-attention**. Woah!!! is this even a term? Yes, if you pay enough attention to what you think and do ;)

**NOTE**: The above discussion was to show you the advanced version of Attention called self attention. We'll use the simple attention. If you want to know more the paper [Attention is all you need](), then you have to refer to very awesome blogs given below:
1. [Pytorch's seq2seq tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb)
2. [Jay Alammar's highly intutive and most referenced blog](http://jalammar.github.io/illustrated-transformer/)
3. [My Favourite blog on the paper](https://glassboxmedicine.com/2019/08/15/the-transformer-attention-is-all-you-need/)
4. [Paper Dissected by mlexplained](https://mlexplained.com/2017/12/29/attention-is-all-you-need-explained/#:~:text=The%20attention%20weight%20can%20be,simple%20feed%2Dforward%20neural%20network.&text=Attention%20basically%20gives%20the%20decoder,choose%20what%20information%20to%20use)

# Workings
To get the illustrated knowledge of working of attention, please refer to [this very intutive blog](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3).

Let us look at the image given below
<img src="https://content.iospress.com/media/ifs/2020/38-2/ifs-38-2-ifs191257/ifs-38-ifs191257-g002.jpg?width=350">

So in a `Many input, Many-output` style scenario, instead of loking at the current input and the previous hidden state like in any RNN type of architecture, it looks at all the instances. In simple terms, it looks at all the words in the sentence everytime and predicts that **if I have to predict a new `y_out`, then which words do I have to look for the most and bring out the most relatable output**. 
So it finds the *Weight* and *Context Vector* at each time step to decide what should we be outputting at a given time step.

We can add Attention Layer to support LSTM layer for a very good Encoder-Decoder Architecture for Machine Translation as shown below:
<img src="https://i.ytimg.com/vi/p3jmVkUMMuw/maxresdefault.jpg" width="550px">



# Imports

Please don't get intimidated by the size of imports. Some are used and some are not given different scenarios.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

from keras import backend as K
from keras import initializers, regularizers, constraints

from keras.layers import Dense, Input, LSTM, Bidirectional,Dropout, Embedding, BatchNormalization, Layer
from keras.models import Model
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from keras.initializers import Constant
from keras.layers.merge import add
from keras.optimizers import Adam

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

# Set Defaults

In [ ]:
seed = 13 # reproducible results

np.random.seed(seed) # numpy seed

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

signs = list(punctuation) # special characters 
stop_words = list(stopwords.words('english')) # stop words like `a,an,the,or,at` etc

wordnet_lemmatizer = WordNetLemmatizer() # make the word cooking,cooks,cooked -> cook (in ideal case)

# Proprecess Data
There might be anomalies with the data we have. We have to remove those impurities so that it can makes our model faster and those impurities do not provide un necessary parameters to the model.

**NOTE**: Cleaning and preprocessing methods are different for every data given What we need, what we want to achieve and what kind of impurities we have. 

In [ ]:
df = pd.read_csv('/kaggle/input/bbc-fulltext-and-category/bbc-text.csv')
df.head()

In [ ]:
df['category'].value_counts().plot(kind='pie',autopct='%.2f%%') # almost equally distributed

## Clean Data

In [ ]:
def clean_articles(df):
    '''
    Method to clean the existing DataFrame
    '''

    # remove all the new lines and spaces if there are any : - \r and \n
    print('Step 1: Replacing...')
    
    df['text'] = df['text'].str.replace("\r", " ")
    df['text'] = df['text'].str.replace("\n", " ")
    df['text'] = df['text'].str.replace("    ", " ")
    df['text'] = df['text'].str.replace('"', '') # remove double quotes
    df['text'] = df['text'].str.lower()  # make all the words as lower case
    
    for sign in signs:
        df['text'] = df['text'].str.replace(sign, '') # remove any special punctuations
        
    # remove Deshwal's the trailing s as it does not add any information in classification
    df['text'] = df['text'].str.replace("'s", "")


    print('Step 2: Lemmatizing......')
    
    nrows = len(df)
    lemmatized_text_list = []

    for row in range(nrows):
        lemmatized_list = [] # Create an empty list containing lemmatized words
        text = df.loc[row]['text'] # Save the text and its words into an object
        text_words = text.split(" ")

        for word in text_words:  # Iterate through every word to lemmatize
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        lemmatized_text = " ".join(lemmatized_list)  # Join the list to get a string
        lemmatized_text_list.append(lemmatized_text) # Append to the list containing the texts

    df['text'] = lemmatized_text_list
    
    
    print('Step 3: Removing Stop Words....')
    
    for stop_word in stop_words:
        re_sw = r"\b" + stop_word + r"\b"
        df['text'] = df['text'].str.replace(re_sw, '')


In [ ]:
clean_articles(df)

## Tokenization

In [ ]:
def tokenize_articles(df):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df.text)
    df['words'] = tokenizer.texts_to_sequences(df.text)
    
    return tokenizer

In [ ]:
tokenizer = tokenize_articles(df)

df['article_length'] = df.words.apply(lambda i: len(i)) # if article length is less than 10, drop it
df = df[df['article_length']>=10]

In [ ]:
df.article_length.describe()

## Padding

In [ ]:
maxlen = 275 # 75 percentile is 273
X = list(sequence.pad_sequences(df.words, maxlen=maxlen)) # add padding to the short length articles

df['encoded_cat'] = LabelEncoder().fit_transform(df['category']) # convert category as 0,1,2,3,4

# Glove embedding
If you want to know what are embeddings and why are they used and how come they are useful, please find the [**Embedding** part of this Kernel](https://www.kaggle.com/deshwalmahesh/nlp-beginner-1-rnn-lstm-gru-embeddings-glove) for a very intutive and real world examples of embeddings.

In [ ]:
word_index = tokenizer.word_index

EMBEDDING_DIM = 100

embeddings_index = {}
with open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f'Unique tokens: {len(word_index)}')
print(f'Total Word Vectors: {len(embeddings_index)}')

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Split Data

In [ ]:
X = np.array(X)
Y = np_utils.to_categorical(df['encoded_cat'].tolist())

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

# Attention Layer

In [ ]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

# Custom Metrics
**Accuracy** is a very decieving metric in terms of interpreting results. If I started talkimg about how it'll take a whole Kernel for this but in simplest terms, let us suppose you have a biased data of `90:10` of 100 images out of which 80 are Non-Spam and 20 are spams. Supposed you trained a model on this data and fount out that you have excellent `accuracy` of `90%`. When you looked at the classified data from model, you found out that it could not detect a single spam. How come?? 

Becuse even if it classified 100 images as `Non-Spam`, it was correct 90% of the times. Can you relate?? **A broken watch is also correct twice a day**. That's what happened.


So we need to find out `Recall`, `Precision` and Harmonic mean of both called `F-1 score`. Because these metrices are not given by Keras, we have to make these metrices.

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Build Attention Model

In [ ]:
embedding_layer = Embedding(len(word_index)+1,EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),input_length=maxlen,
                            trainable=False) # do not train as these are GloVe pre trained

In [ ]:
inp = Input(shape=(maxlen,), dtype='int32')
embedding= embedding_layer(inp)
attention_out = Attention()(embedding)
flat = Dense(254, activation='relu')(attention_out)
flat = Dropout(0.47)(flat)
flat = BatchNormalization()(flat)
out = Dense(5,activation='softmax')(flat)

AttentionModel = Model(inputs=inp, outputs=out)
AttentionModel.compile(loss='categorical_crossentropy', optimizer='adam', 
                       metrics=['acc',recall_m,precision_m,f1_m])

AttentionModel.summary()

In [ ]:
mcp = ModelCheckpoint(filepath='/kaggle/working/best_weights.h5',verbose=1,save_best_only=True,
                      save_weights_only=True)
es = EarlyStopping(min_delta=0.01,patience=2,verbose=1)
rlp = ReduceLROnPlateau(factor=0.005,patience=1,verbose=1,min_delta=0.001,min_lr=1e-6)

callbacks = [mcp,es,rlp]

## Training

In [ ]:
training_history = AttentionModel.fit(x_train,y_train,batch_size=32,epochs=10,
                                      validation_data=(x_val, y_val),callbacks=callbacks)

# Attention on top of LSTM
Let us try to add a LSTM layer with Attention to see what happens. It surely will make the model slower.

In [ ]:
inp = Input(shape=(maxlen,), dtype='int32')
embedding= embedding_layer(inp)
lstm_out = LSTM(254, dropout=0.27, recurrent_dropout=0.25, return_sequences=True)(embedding)
x = Dropout(0.49)(lstm_out)
attention_out = Attention()(x)
flat = Dense(512, activation='relu')(attention_out)
flat = Dropout(0.69)(flat)
flat = BatchNormalization()(flat)
out = Dense(5,activation='softmax')(flat)

WithLSTM = Model(inputs=inp, outputs=out)
WithLSTM.compile(loss='categorical_crossentropy', optimizer='adam', 
                       metrics=['acc',recall_m,precision_m,f1_m])

WithLSTM.summary()

In [ ]:
training_history2 = WithLSTM.fit(x_train,y_train,batch_size=64,epochs=5,
                                      validation_data=(x_val, y_val),callbacks=callbacks)

# Thank You
As you have seen that our models are producing good results with the methods. You can also try to make implement some new ideas by applyting `Bidirectional` layer wrapped around LSTM or GRU with the Attention Layer to find out how it goes. To improve model, you can change epochs, batch sizes, number of layers, number of neurons, dropout values and much more.

Hope you had some good understanding of the topic. Please let me know in case otherwise for corrections or improvements.